In [1]:
%load_ext autoreload
%autoreload 2
from tf_activation import DeepGraph
from tf_activation.models import mnist_cff as mnist_model

import math
import random
import os

import networkx as nx
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.metrics import r2_score
%matplotlib inline
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist_map = {
    0: 3,
    1: 2,
    2: 1,
    3: 18,
    4: 4,
    5: 8,
    6: 11,
    7: 0,
    8: 61,
    9: 7
}

SAVE_PATH = '../logdir/models'
ADV_PATH = '../logdir/adversaries'
ELITE_PATH = '../logdir/elites'
DATA_PATH = '../logdir/data'
SAVE_FIG_LOC = '../logdir/figures'


mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allocator_type = 'BFC'
config.log_device_placement = True

persistence_module = tf.load_op_library('/home/tgebhart/python/tensorflow/bazel-bin/tensorflow/core/user_ops/nn_graph_persistence.so')

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
im_1 = mnist.test.images[mnist_map[1]]
im_2 = mnist.test.images[mnist_map[2]]
im_rand = np.random.rand(im_2.shape[0])

steps = 200

columns = ['in_distance_1', 'in_distance_2', 'per_distance_1', 'per_distance_2', 'cross_entropy_1',
           'cross_entropy_2', 'y_conv_1', 'y_conv_2', 'accuracy_1', 'accuracy_2']
index = range(steps)
test_df = pd.DataFrame(index=index, columns=columns)

In [4]:
def create_path(start_im, end_im, steps=100):
    ret = []
    t = np.divide(end_im - start_im, steps)
    for i in range(1,steps+1):
        ret.append(start_im + i*t)
    return ret
    

In [5]:
path = create_path(im_1, im_rand) + create_path(im_rand, im_2)

In [ ]:
# setup model
with tf.device('/cpu:0'):
    # Create the model
    x = tf.placeholder(tf.float32, [None, 784])

    # Define loss and optimizer
    y_ = tf.placeholder(tf.float32, [None, 10])

    # Build the graph for the deep net
    net, keep_prob = mnist_model.deepnn(x)

    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=net['y_conv']))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(net['y_conv'], 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    saver = tf.train.Saver()

In [ ]:
# run network
test_labels = np.stack((mnist.test.labels[mnist_map[1]],mnist.test.labels[mnist_map[2]]))
p = 95
with tf.Session(config=config) as sess:

    saver.restore(sess, os.path.join(SAVE_PATH, 'mnist_cff_2000.ckpt'))

    test_inputs_1 = np.stack((im_1, path[0]))
    test_inputs_2 = np.stack((im_2, path[0]))

    percentiles = persistence_module.layerwise_percentile([net['input'],
                                                        net['W_conv1'],
                                                        net['h_conv1'],
                                                        net['h_conv1'],
                                                        net['W_fc1'],
                                                        net['h_fc1'],
                                                        net['h_fc1_drop'],
                                                        net['W_fc2'],
                                                        net['y_conv']],
                                                        [0, 1, 2, 2, 1, 4, 4, 1, 4],
                                                        [p,p,p])

    ps1_1 = percentiles.eval(feed_dict={x: test_inputs_1[:1], keep_prob:1.0})
    ps1_2 = percentiles.eval(feed_dict={x: test_inputs_2[:1], keep_prob:1.0})
    
    for i in range(len(path)):
        
        test_inputs_1 = np.stack((im_1, path[i]))
        test_inputs_2 = np.stack((im_2, path[i]))
        in_distance_1 = np.linalg.norm(im_1 - path[i], ord=2)
        in_distance_2 = np.linalg.norm(im_2 - path[i], ord=2)

        ps2 = percentiles.eval(feed_dict={x: test_inputs_1[1:], keep_prob:1.0})

        result_1 = persistence_module.wasserstein_distance([net['input'],
                                                        net['W_conv1'],
                                                        net['h_conv1'],
                                                        net['h_conv1'],
                                                        net['W_fc1'],
                                                        net['h_fc1'],
                                                        net['h_fc1_drop'],
                                                        net['W_fc2'],
                                                        net['y_conv']],
                                                        [0, 1, 2, 2, 1, 4, 4, 1, 4],
                                                        np.stack((ps1_1, ps2))
                                                        )
        
        result_2 = persistence_module.wasserstein_distance([net['input'],
                                                        net['W_conv1'],
                                                        net['h_conv1'],
                                                        net['h_conv1'],
                                                        net['W_fc1'],
                                                        net['h_fc1'],
                                                        net['h_fc1_drop'],
                                                        net['W_fc2'],
                                                        net['y_conv']],
                                                        [0, 1, 2, 2, 1, 4, 4, 1, 4],
                                                        np.stack((ps1_2, ps2))
                                                        )


        per_distance_1 = result_1.eval(feed_dict={x: test_inputs_1, keep_prob:1.0})
        per_distance_2 = result_2.eval(feed_dict={x: test_inputs_2, keep_prob:1.0})
        print('Step: ', i)
        
        ce_1 = cross_entropy.eval(feed_dict={x:test_inputs_1[1:], y_:test_labels[:1], keep_prob:1.0})
        ce_2 = cross_entropy.eval(feed_dict={x:test_inputs_2[1:], y_:test_labels[1:], keep_prob:1.0})
        y_conv_1 = sess.run(net['y_conv'], feed_dict={x:test_inputs_1[1:], keep_prob:1.0})
        y_conv_2 = sess.run(net['y_conv'], feed_dict={x:test_inputs_2[1:], keep_prob:1.0})
        acc_1 = accuracy.eval(feed_dict={x:test_inputs_1[1:], y_:test_labels[:1], keep_prob:1})
        acc_2 = accuracy.eval(feed_dict={x:test_inputs_2[1:], y_:test_labels[1:], keep_prob:1})
        y_conv_1 = y_conv_1 / np.linalg.norm(y_conv_1)
        y_conv_2 = y_conv_2 / np.linalg.norm(y_conv_2)

        test_df.loc[i, 'in_distance_1'] = in_distance_1
        test_df.loc[i, 'per_distance_1'] = per_distance_1[0]
        test_df.loc[i, 'cross_entropy_1'] = ce_1
        test_df.loc[i, 'y_conv_1'] = y_conv_1[0,np.argmax(test_labels[0], axis=0)]
        test_df.loc[i, 'accuracy_1'] = acc_1
        
        test_df.loc[i, 'in_distance_2'] = in_distance_2
        test_df.loc[i, 'per_distance_2'] = per_distance_2[0]
        test_df.loc[i, 'cross_entropy_2'] = ce_2
        test_df.loc[i, 'y_conv_2'] = y_conv_2[0,np.argmax(test_labels[1], axis=0)]
        test_df.loc[i, 'accuracy_2'] = acc_2
        

INFO:tensorflow:Restoring parameters from ../logdir/models/mnist_cff_2000.ckpt


In [ ]:
test_df.head()

In [ ]:
def regress(X, y, xlab, ylab, color_vals='b'):
    
    print(X.shape,y.shape)
    
    # Split the data into training/testing sets
    X_train = X[:-20]
    X_test = X[-20:]

    # Split the targets into training/testing sets
    y_train = y[:-20]
    y_test = y[-20:]

    # Create linear regression object
    regr = linear_model.LinearRegression()

    # Train the model using the training sets
    regr.fit(X_train, y_train)

    # The coefficients
    print('Coefficients: ', regr.coef_)
    # The mean squared error
    print("Mean squared error: %.2f"
          % np.mean((regr.predict(X_test) - y_test) ** 2))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % regr.score(X_test, y_test))
    print('R^2: %.2f' % r2_score(y_test, regr.predict(X_test)))
    
    fig, ax = plt.subplots()
    X_min = np.min(X)
    X_max = np.max(X)
    y_min = np.min(y)
    y_max = np.max(y)
    # Plot outputs
    
    ax.scatter(X, y,  color=color_vals, s=7)
#     ax.plot(X, regr.predict(X), color='blue', linewidth=3)


    ax.set_xlim(X_min, X_max)
    ax.set_ylim(y_min, y_max)
    
    plt.xlabel(xlab)
    plt.ylabel(ylab)
#     plt.title('{} vs {}'.format(xlab, ylab))
    plt.show()
    fig.savefig(os.path.join(SAVE_FIG_LOC, xlab + '_' + ylab + '.svg'), 
                format='svg', dpi=1200, bbox_inches='tight')


In [ ]:
color_vals = ['#4286f4' if i < 100 else '#f4734b' for i in range(200)]

In [ ]:
X = test_df['in_distance_1'].as_matrix()
X = X.reshape((X.shape[0], 1))
y = test_df['per_distance_1'].as_matrix()
regress(X, y, 'Input Distance 1', 'Persistence Distance 1', color_vals='#4286f4')

In [ ]:
X = test_df['in_distance_1'].as_matrix()
X = X.reshape((X.shape[0], 1))
y = test_df['per_distance_1'].as_matrix()
regress(X[:26], y[:26], 'Input Distance 1', 'Persistence Distance 1', color_vals='#4286f4')

In [ ]:
X = np.array(test_df.index)
X = X.reshape((X.shape[0], 1))
regress(X, y, 'Step', 'Persistence Distance 1', color_vals='#4286f4')

In [ ]:
X = test_df['cross_entropy_1'].as_matrix()
X = X.reshape((X.shape[0], 1))
regress(X, y, 'Cross-Entropy 1', 'Persistence Distance 1', color_vals='#4286f4')

In [ ]:
X = test_df['cross_entropy_1'].as_matrix()
X = X.reshape((X.shape[0], 1))
regress(X[:26], y[:26], 'Cross-Entropy 1', 'Persistence Distance 1', color_vals='#4286f4')

In [ ]:
X = test_df['y_conv_1'].as_matrix()
X = X.reshape((X.shape[0], 1))
regress(X, y, 'Correct Class Probability 1', 'Persistence Distance 1', color_vals='#4286f4')

In [ ]:
X = test_df['y_conv_1'].as_matrix()
X = X.reshape((X.shape[0], 1))
regress(X[:26], y[:26], 'Correct Class Probability 1', 'Persistence Distance 1', color_vals='#4286f4')

In [ ]:
X = test_df['in_distance_2'].as_matrix()
X = X.reshape((X.shape[0], 1))
y = test_df['per_distance_2'].as_matrix()
regress(X, y, 'Input Distance 2', 'Persistence Distance 2', color_vals='#f4564a')

In [ ]:
X = test_df['cross_entropy_2'].as_matrix()
X = X.reshape((X.shape[0], 1))
regress(X, y, 'Cross-Entropy 2', 'Persistence Distance 2', color_vals='#f4564a')

In [ ]:
X = test_df['y_conv_2'].as_matrix()
X = X.reshape((X.shape[0], 1))
regress(X, y, 'Correct Class Probability 2', 'Persistence Distance 2', color_vals='#f4564a')

In [ ]:
X = np.array(test_df.index)
X = X.reshape((X.shape[0], 1))
regress(X, y, 'Step', 'Persistence Distance 2', color_vals='#f4564a')

In [ ]:
X = np.append(test_df['in_distance_1'].as_matrix(), test_df['in_distance_2'].as_matrix(), axis=0)
X = X.reshape((X.shape[0], 1))
y = np.append(test_df['per_distance_1'].as_matrix(), test_df['per_distance_2'].as_matrix(), axis=0)
color_vals = ['#4286f4' if i < X.shape[0]//2 else '#f4564a' for i in range(X.shape[0])]
regress(X, y, 'Persistence Distance', color_vals=color_vals)


In [ ]:
X = np.append(np.array(test_df.index), np.array(test_df.index), axis=0)
X = X.reshape((X.shape[0], 1))
y = np.append(test_df['per_distance_1'].as_matrix(), test_df['per_distance_2'].as_matrix(), axis=0)
color_vals = ['#4286f4' if i < X.shape[0]//2 else '#f4564a' for i in range(X.shape[0])]
regress(X, y, 'Step', 'Persistence Distance', color_vals=color_vals)